In [22]:
from sec_api import ExtractorApi, XbrlApi
import pandas as pd
from IPython.display import display, HTML
import io

xbrlApi = XbrlApi("2b48b80bbfb42bc4ac67b3812ca3f1a00f3f25440b39e612134d9445f250fa03")
extractorapi = ExtractorApi("2b48b80bbfb42bc4ac67b3812ca3f1a00f3f25440b39e612134d9445f250fa03")

URLS = {
    "url_10k_coinbase_2021_mar_31": "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001679788/000167978821000010/coin-20210331.htm",
    "url_10k_coinbase_2021_june_30": "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001679788/000167978821000026/coin-20210630.htm",
    "url_10q_coinbase_q1_2021_sept_30": "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001679788/000167978821000071/coin-20210930.htm",
    "url_10q_coinbase_q2_2022_mar_31": "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001679788/000167978822000048/coin-20220331.htm",
    "url_10q_coinbase_q3_2022_june_30": "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001679788/000167978822000085/coin-20220630.htm", 
    "url_10q_coinbase_q4_2022_sept_30": "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001679788/000167978822000116/coin-20220930.htm",
}

for key, url in URLS.items():
    xbrl_json = xbrlApi.xbrl_to_json(htm_url=url)
    print(xbrl_json.keys())

# summary_tables = {}
# # extract text section "Item 1 - Business" from 10-K
# for key, url in URLS.items():
#     item_1_html = extractorapi.get_section(url, 'part1item2', 'text')
#     # print(item_1_html)
#     tables = pd.read_html(io.StringIO(item_1_html), header=1)
#     df = tables[0]
#     # new_header = df.iloc[0] + df.iloc[1]
#     # df = df[2:]
#     # df.columns = new_header
#     mask = (df.iloc[1:, :].isna()).all(axis=0)
#     financial_statements = df.drop(df.columns[mask], axis=1).fillna('')
#     summary_tables[key] = financial_statements
#     display(financial_statements)

# display(HTML(item_1_html[0:150000]))

Exception: API error: 429 - {"status":429,"error":"You send a lot of requests. We like that. But you exceeded the free query limit of 100 requests. Upgrade your account to get unlimited access. Visit sec-api.io for more."}

In [53]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import html2text
import tempfile
import pandas as pd
import io

CIK = "0001679788"
headers = {"User-Agent":"tushitasingh05@gmail.com"}

def merge_first_n_rows_as_header(df, n=3):
    # Step 1: Take the first n rows as potential headers
    header_rows = df.iloc[:n].astype(str)
    
    # Step 2: Merge them into single column names
    merged_header = (
        header_rows.apply(lambda x: ' '.join([i for i in x if i != 'nan']).strip(), axis=0)
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'Unnamed.*', '', regex=True)
    )
    
    # Step 3: Apply merged header
    df.columns = merged_header
    
    # Step 4: Drop the used header rows
    df = df.iloc[n:].reset_index(drop=True)
    
    # Step 5: Drop duplicate columns (those that got merged to same name)
    # df = df.loc[:, ~df.columns.duplicated()]
    left = df.iloc[:, 1]  # keep first few descriptive cols
    right = df.iloc[:, 3:].dropna(axis=1, how='any')  # clean numeric cols
    mask_dollar = right.apply(lambda col: col.astype(str).str.contains(r'\$|\%', regex=True).any())
    right = right.loc[:, ~mask_dollar]
    right = right.T.drop_duplicates().T

    df = pd.concat([left, right], axis=1)
    
    return df


def fetch_xbrl(url):
    r = requests.get(url, headers=headers)
    raw_xbrl_content = r.text

     # Step 2: Save HTML to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as tmp_file:
        tmp_file.write(raw_xbrl_content.encode('utf-8'))
        local_html_path = tmp_file.name

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get("file://" + local_html_path)
    rendered_html = driver.page_source
    driver.quit()

    # Step 4: Extract content sections and tables
    soup = BeautifulSoup(rendered_html, "html.parser")

    # Optional cleanup: remove scripts, styles, etc.
    for tag in soup(["script", "style"]):
        tag.extract()

    # Convert HTML to Markdown
    markdown_text = html2text.html2text(str(soup))
    tables = pd.read_html(io.StringIO(rendered_html))

    # for number, tbl in enumerate(tables):
    #     print("----------", number, "----------------------------------")
    #     tbl = clean_sec_table(tbl)
    #     print(tbl)

    key_business_metrics = None
    for i, tbl in enumerate(tables):
        if tbl.astype(str).apply(lambda x: x.str.contains("Verified Users", case=False, na=False)).any().any():
            print(f"✅ Found 'Verified Users' in table index {i}")
            key_business_metrics = tbl
            break

    if key_business_metrics is None:
        print("❌ Could not find a table containing 'Verified Users'")
        return None
    
    key_business_metrics = merge_first_n_rows_as_header(key_business_metrics, n=4)
    display(key_business_metrics)
    return key_business_metrics

URLS = {
    "url_10k_coinbase_q1_2021_03_31": "https://www.sec.gov/Archives/edgar/data/0001679788/000167978821000010/coin-20210331.htm",
    "url_10k_coinbase_q2_2021_06_30": "https://www.sec.gov/Archives/edgar/data/0001679788/000167978821000026/coin-20210630.htm",
    "url_10q_coinbase_q3_2021_09_30": "https://www.sec.gov/Archives/edgar/data/0001679788/000167978821000071/coin-20210930.htm",
    "url_10q_coinbase_q1_2022_03_31": "https://www.sec.gov/Archives/edgar/data/0001679788/000167978822000048/coin-20220331.htm",
    "url_10q_coinbase_q2_2022_06_30": "https://www.sec.gov/Archives/edgar/data/0001679788/000167978822000085/coin-20220630.htm", 
    "url_10q_coinbase_q3_2022_09_30": "https://www.sec.gov/Archives/edgar/data/0001679788/000167978822000116/coin-20220930.htm",
}

clean_tables = {}
for key, url in URLS.items():
    df_rev = fetch_xbrl(url)
    clean_tables[key] = df_rev

✅ Found 'Verified Users' in table index 47


,,"As of or for the Quarter Ended March 31, 2021 (in millions, except percentages)","As of or for the Quarter Ended March 31, 2020 (in millions, except percentages)","% Change % Change (in millions, except percentages)"
0,Verified Users,56,34,65
1,MTUs,6.1,1.3,369
2,Assets on Platform,222748,16896,1218
3,Trading Volume,334737,29827,1022
4,Net income,771,32,2309
5,Adjusted EBITDA(1),1117,55,1931


✅ Found 'Verified Users' in table index 50


,,"Three Months Ended June 30, 2021","Three Months Ended June 30, 2020",% Change % Change,"Six Months Ended June 30, 2021","Six Months Ended June 30, 2020",% Change % Change
0,Verified Users (in millions),68,36,89,68,36,89
1,MTUs (in millions),8.8,1.5,487,7.5,1.4,436
2,Assets on Platform (in billions),180,26,592,180,26,592
3,Trading Volume (in billions),462,28,1550,796,59,1249
4,Net income (in millions),1606,32,4919,2378,64,3616
5,Adjusted EBITDA(1) (in millions),1150,61,1785,2267,116,1854


✅ Found 'Verified Users' in table index 55


...

,,"Three Months Ended March 31, 2022","Three Months Ended March 31, 2021",% Change % Change
0,Verified Users(1) (in millions),98,54,81
1,MTUs (in millions),9.2,6.1,51
2,Assets on Platform (in billions),256,223,15
3,Trading Volume (in billions),309,335,(8)
4,Net (loss) income (in millions),(430),771,(156)
5,Adjusted EBITDA(2) (in millions),20,1117,(98)


✅ Found 'Verified Users' in table index 56


,,"Three Months Ended June 30, 2022","Three Months Ended June 30, 2021",% Change % Change,"Six Months Ended June 30, 2022","Six Months Ended June 30, 2021",% Change % Change
0,Verified Users (in millions),103,68,51,103,68,51
1,MTUs(1) (in millions),9.0,8.8,2,9.1,7.5,21
2,Assets on Platform (in billions),96,180,(47),96,180,(47)
3,Trading Volume (in billions),217,462,(53),526,796,(34)
4,Net (loss) income (in millions),"(1,094)",1606,(168),"(1,523)",2378,(164)
5,Adjusted EBITDA(2) (in millions),(151),1150,(113),(131),2267,(106)


✅ Found 'Verified Users' in table index 61


,,"Three Months Ended September 30, 2022","Three Months Ended September 30, 2021",% Change % Change,"Nine Months Ended September 30, 2022","Nine Months Ended September 30, 2021",% Change % Change
0,Verified Users (in millions),108,73,48,108,73,48
1,MTUs(1) (in millions),8.5,7.3,16,8.9,7.3,22
2,Assets on Platform (in billions),101,255,(60),101,255,(60)
3,Trading Volume (in billions),159,327,(51),685,1124,(39)
4,Net (loss) income (in millions),(545),406,(234),"(2,068)",2784,(174)
5,Adjusted EBITDA(2) (in millions),(116),618,(119),(247),2885,(109)


In [55]:
import re
metric_map = {
    "verified users": "verified_users",
    "mtus": "monthly_transacting_users",
    "assets on platform": "assets_on_platform",
    "trading volume": "trading_volume",
    "net income": "net_income",
    "adjusted ebitda": "adjusted_ebitda"
}

def _standardize_name(name):
    name_lower = name.lower()
    for key, val in metric_map.items():
        if key in name_lower:
            return val
    return name_lower.replace(" ", "_")

def clean_key_metrics(df, period_end, filing_type):

    first_col = df.columns[0]
    df[first_col] = df[first_col].apply(lambda x: _standardize_name(str(x)))
    df.rename(columns={first_col: "metric"}, inplace=True)

    # Step 2: clean column headers
    clean_cols = []
    for c in df.columns:
        c = re.sub(r'\(.*?\)', '', c).strip()
        c = re.sub(r'As of or for the (Quarter|Three Months|Six Months) Ended ', '', c).strip()
        # c = re.sub(r'(Three Months|Six Months|Nine Months) Ended ', '', c).strip()
        c = re.sub(r'% Change % Change', '% Change', c).strip()
        c = c.replace(',', '')
        c = c.lower()
        clean_cols.append(c)
    df.columns = clean_cols
    
    # # Step 3: Convert numeric text to float
    # for col in df.columns[1:]:
    #     df[col] = (
    #         df[col]
    #         .astype(str)
    #         .str.replace(r"[\$,()%]", "", regex=True)
    #         .replace("", pd.NA)
    #         .astype(float)
    #     )

    # Step 4: Add period info
    df["period_end"] = period_end
    df["filing_type"] = filing_type
    return df

cleaned_list = []

for key, df in clean_tables.items():
    period_end, filing_type = "-".join((key.split("_"))[4:7]), "10Q"
    cleaned_df = clean_key_metrics(df, period_end, filing_type)
    # Remove duplicate columns before appending
    cleaned_list.append(cleaned_df) 
    display(cleaned_df)

all_metrics = cleaned_list[0]

for next_df in cleaned_list[1:]:
    all_metrics = pd.merge(all_metrics, next_df, on="metric", how="outer")
display(all_metrics)

,metric,march 31 2021,march 31 2020,% change,period_end,filing_type
0,verified_users,56,34,65,2021-03-31,10Q
1,monthly_transacting_users,6.1,1.3,369,2021-03-31,10Q
2,assets_on_platform,222748,16896,1218,2021-03-31,10Q
3,trading_volume,334737,29827,1022,2021-03-31,10Q
4,net_income,771,32,2309,2021-03-31,10Q
5,adjusted_ebitda,1117,55,1931,2021-03-31,10Q


,metric,three months ended june 30 2021,three months ended june 30 2020,% change,six months ended june 30 2021,six months ended june 30 2020,% change,period_end,filing_type
0,verified_users,68,36,89,68,36,89,2021-06-30,10Q
1,monthly_transacting_users,8.8,1.5,487,7.5,1.4,436,2021-06-30,10Q
2,assets_on_platform,180,26,592,180,26,592,2021-06-30,10Q
3,trading_volume,462,28,1550,796,59,1249,2021-06-30,10Q
4,net_income,1606,32,4919,2378,64,3616,2021-06-30,10Q
5,adjusted_ebitda,1150,61,1785,2267,116,1854,2021-06-30,10Q


,metric,three months ended september 30 2021,three months ended september 30 2020,% change,nine months ended september 30 2021,nine months ended september 30 2020,% change,period_end,filing_type
0,verified_users,73,39,87,73,39,87,2021-09-30,10Q
1,monthly_transacting_users,7.4,2.1,252,7.4,1.6,363,2021-09-30,10Q
2,assets_on_platform,255,36,608,255,36,608,2021-09-30,10Q
3,trading_volume,327,45,627,1124,104,981,2021-09-30,10Q
4,net_income,406,81,401,2784,146,1807,2021-09-30,10Q
5,adjusted_ebitda,618,123,402,2885,240,1102,2021-09-30,10Q


,metric,three months ended march 31 2022,three months ended march 31 2021,% change,period_end,filing_type
0,verified_users,98,54,81,2022-03-31,10Q
1,monthly_transacting_users,9.2,6.1,51,2022-03-31,10Q
2,assets_on_platform,256,223,15,2022-03-31,10Q
3,trading_volume,309,335,(8),2022-03-31,10Q
4,net_(loss)_income_(in_millions),(430),771,(156),2022-03-31,10Q
5,adjusted_ebitda,20,1117,(98),2022-03-31,10Q


,metric,three months ended june 30 2022,three months ended june 30 2021,% change,six months ended june 30 2022,six months ended june 30 2021,% change,period_end,filing_type
0,verified_users,103,68,51,103,68,51,2022-06-30,10Q
1,monthly_transacting_users,9.0,8.8,2,9.1,7.5,21,2022-06-30,10Q
2,assets_on_platform,96,180,(47),96,180,(47),2022-06-30,10Q
3,trading_volume,217,462,(53),526,796,(34),2022-06-30,10Q
4,net_(loss)_income_(in_millions),"(1,094)",1606,(168),"(1,523)",2378,(164),2022-06-30,10Q
5,adjusted_ebitda,(151),1150,(113),(131),2267,(106),2022-06-30,10Q


,metric,three months ended september 30 2022,three months ended september 30 2021,% change,nine months ended september 30 2022,nine months ended september 30 2021,% change,period_end,filing_type
0,verified_users,108,73,48,108,73,48,2022-09-30,10Q
1,monthly_transacting_users,8.5,7.3,16,8.9,7.3,22,2022-09-30,10Q
2,assets_on_platform,101,255,(60),101,255,(60),2022-09-30,10Q
3,trading_volume,159,327,(51),685,1124,(39),2022-09-30,10Q
4,net_(loss)_income_(in_millions),(545),406,(234),"(2,068)",2784,(174),2022-09-30,10Q
5,adjusted_ebitda,(116),618,(119),(247),2885,(109),2022-09-30,10Q


MergeError: Passing 'suffixes' which cause duplicate columns {'period_end_x', '% change_x', 'filing_type_x'} is not allowed.